In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
from tensorflow.keras.utils import to_categorical

In [ ]:
# defining the residual block

def residual_block(x, filters, kernel_size=3, stride=1, use_shortcut=False):
    # Shortcut connection
    shortcut = x

    # First convolution
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second convolution
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add shortcut to the main path (skip connection)
    if use_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)

    return x

In [ ]:
# building the resnet

def build_resnet18(input_shape=(32, 32, 3), num_classes=10):
    # Input layer
    input_tensor = tf.keras.Input(shape=input_shape)

    # Initial Convolution
    x = layers.Conv2D(64, 7, strides=2, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, 64, use_shortcut=True)
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2, use_shortcut=True)
    x = residual_block(x, 128)
    x = residual_block(x, 256, stride=2, use_shortcut=True)
    x = residual_block(x, 256)
    x = residual_block(x, 512, stride=2, use_shortcut=True)
    x = residual_block(x, 512)

    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Fully connected layer
    x = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = models.Model(inputs=input_tensor, outputs=x, name='resnet18')

    return model

In [ ]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [ ]:
train_images.shape, train_labels.shape

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
# One-hot encode the labels

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

In [ ]:
# Create ResNet-18 model
resnet18_model = build_resnet18()

# Display the model summary
resnet18_model.summary()

In [ ]:
# Compile the model
resnet18_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [13]:
# Train the model
resnet18_model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_data=(test_images, test_labels)
    )

1563/1563 [==============================] - 1653s 1s/step - loss: 0.7256 - accuracy: 0.7498 - val_loss: 1.2720 - val_accuracy: 0.5717


In [11]:
# using a pretrained model using keras

from tensorflow.keras.applications import ResNet50

model = ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

102967424/102967424 [==============================] - 1s 0us/step


In [15]:
# Evaluate the model on the test set

test_loss, test_acc = resnet18_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc * 100:.2f}%")

313/313 [==============================] - 23s 72ms/step - loss: 1.2720 - accuracy: 0.5717
Test accuracy: 57.17%


That's all